In [1]:
pip install pycaret

## Treinando com julho e agosto e testando com setembro

In [4]:
import pandas as pd
import numpy as np
from pycaret.classification import *
from sklearn.metrics import accuracy_score, classification_report

path = '/content'
data_path_set = f"{path}/set_2024.csv"
data_path_ago = f"{path}/ago_2024.csv"
data_path_jul = f"{path}/jul_2024.csv"

path = '/content'
data_path_set_latent = f"{path}/set_latent_10.csv"
data_path_ago_latent = f"{path}/ago_latent_10.csv"
data_path_jul_latent = f"{path}/jul_latent_10.csv"

set_data = pd.read_csv(data_path_set)
ago_data = pd.read_csv(data_path_ago)
jul_data = pd.read_csv(data_path_jul)

set_data_latent = pd.read_csv(data_path_set_latent)
ago_data_latent = pd.read_csv(data_path_ago_latent)
jul_data_latent = pd.read_csv(data_path_jul_latent)

data_concat = pd.concat([ago_data, jul_data], ignore_index=True)
data_concat_latent = pd.concat([ago_data_latent, jul_data_latent], ignore_index=True)

print("Tamanho do dataset combinado:", data_concat.shape)
print("Número de discos jul e ago:", data_concat['serial_number'].nunique())
data_concat.head()

print("Tamanho do dataset latente combinado:", data_concat_latent.shape)
data_concat_latent.head()

Tamanho do dataset combinado: (3268569, 26)
Número de discos jul e ago: 53011
Tamanho do dataset latente combinado: (2614687, 10)


,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9
0,0.000000,2.032150,0.0,0.0,2.117002,0.0,1.794483,3.087483,3.717370,0.0
1,0.000000,2.026185,0.0,0.0,2.034644,0.0,1.917212,3.185697,3.807883,0.0
2,0.131414,1.750621,0.0,0.0,2.828636,0.0,0.600640,2.322093,2.982116,0.0
3,0.142935,1.656677,0.0,0.0,2.419923,0.0,1.887898,3.119124,3.659531,0.0
4,0.199543,1.665560,0.0,0.0,2.544263,0.0,1.555166,2.873652,3.571856,0.0


In [5]:
# Combinar os rótulos com os dados latentes
data_concat_latent['failure'] = data_concat['failure']
data_concat_latent['serial_number'] = data_concat['serial_number']

# Exibir os dados combinados
print(data_concat_latent)

         Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0         0.000000   2.032150        0.0        0.0   2.117002        0.0   
1         0.000000   2.026185        0.0        0.0   2.034644        0.0   
2         0.131414   1.750621        0.0        0.0   2.828636        0.0   
3         0.142935   1.656677        0.0        0.0   2.419923        0.0   
4         0.199543   1.665560        0.0        0.0   2.544263        0.0   
...            ...        ...        ...        ...        ...        ...   
2614682   0.000000   0.000000        0.0        0.0   0.004658        0.0   
2614683   0.000000   0.000000        0.0        0.0   0.164885        0.0   
2614684   0.000000   0.000000        0.0        0.0   0.544205        0.0   
2614685   0.000000   0.000000        0.0        0.0   0.559638        0.0   
2614686   0.000000   0.000000        0.0        0.0   0.106049        0.0   

         Feature_6  Feature_7  Feature_8  Feature_9  failure serial_number 

In [6]:
# Resumo dados por mês
data_concat['date'] = pd.to_datetime(data_concat['date'])
data_concat['month'] = data_concat['date'].dt.month

df_month = data_concat.groupby('month').agg({'date':['min','max'], 'serial_number':['nunique', 'count'], 'failure':'sum'}).reset_index()
df_month.columns = ['month', 'first_date', 'last_date', 'disks', 'records', 'failure_records']
df_month['failure_rate'] = df_month['failure_records']/df_month['records']*100
df_month.sort_values('failure_rate', ascending=False)

,month,first_date,last_date,disks,records,failure_records,failure_rate
0,7,2024-07-01,2024-07-31,52884,1634170,72,0.004406
1,8,2024-08-01,2024-08-31,52937,1634399,66,0.004038


In [10]:
feature_cols = [c for c in data_concat_latent.columns if c.startswith('Feature')]
std_feature_cols = [np.std(data_concat_latent[c]) for c in feature_cols]
columns_to_use = [c for c, std in zip(feature_cols, std_feature_cols) if std > 0] + ['failure']

# Função para aplicar por grupo
def escolher_linha(grupo):
    if (grupo['failure'] == 1).any():  # Se houver alguma linha com failure == 1
        return grupo[grupo['failure'] == 1].iloc[0]  # Seleciona a primeira linha com failure == 1
    else:
        return grupo.sample(1).iloc[0]  # Seleciona uma linha aleatória

# Aplicar a função por grupo
data_disk = data_concat_latent.groupby('serial_number', group_keys=False).apply(escolher_linha).reset_index(drop=True)
print(data_disk['serial_number'].nunique()) #52835
data_disk = data_disk[columns_to_use]
print(data_disk.shape)
print(data_disk['failure'].value_counts())
data_disk.head()

53009
(53009, 9)
failure
0    52903
1      106
Name: count, dtype: int64


,Feature_0,Feature_1,Feature_2,Feature_4,Feature_6,Feature_7,Feature_8,Feature_9,failure
0,0.000000,0.000000,0.0,0.431247,2.778082,0.000000,1.211041,3.024299,0
1,0.136966,1.948494,0.0,2.666475,0.947908,2.363504,3.425669,0.000000,0
2,0.143017,2.011970,0.0,2.372897,1.367424,2.629718,3.829142,0.000000,0
3,0.000000,2.697626,0.0,2.106703,1.782331,3.080728,2.826706,0.000000,1
4,0.204914,1.668005,0.0,2.217360,1.908147,3.190034,3.899990,0.000000,0


In [11]:
from imblearn.combine import SMOTEENN

X = data_disk.drop(columns='failure')
y = data_disk['failure']
print('Tamanho dataset por disco:', len(data_disk))
# ['auto', 0.001, 0.01, 0.1]:
rus = smote_enn = SMOTEENN(random_state=42, sampling_strategy='auto')
X_resampled, y_resampled = smote_enn.fit_resample(X.fillna(0), y)

print('Tamanho dataset com oversample:', len(X_resampled))
print('Variável resposta após oversample:', y_resampled.value_counts())

data_resampled = pd.concat([X_resampled, y_resampled], axis=1)
print(data_resampled.shape)
data_resampled.head()

Tamanho dataset por disco: 53009
Tamanho dataset com oversample: 103516
Variável resposta após oversample: failure
1    52437
0    51079
Name: count, dtype: int64
(103516, 9)


,Feature_0,Feature_1,Feature_2,Feature_4,Feature_6,Feature_7,Feature_8,Feature_9,failure
0,0.000000,0.000000,0.0,0.431247,2.778082,0.000000,1.211041,3.024299,0
1,0.136966,1.948494,0.0,2.666475,0.947908,2.363504,3.425669,0.000000,0
2,0.143017,2.011970,0.0,2.372897,1.367424,2.629718,3.829142,0.000000,0
3,0.204914,1.668005,0.0,2.217360,1.908147,3.190034,3.899990,0.000000,0
4,0.000000,0.000000,0.0,0.486292,2.464118,0.000000,1.349250,2.468001,0


In [13]:
train_resampled_data = data_resampled
set_data_latent['failure'] = set_data['failure']
test_resampled_data = set_data_latent[columns_to_use]

print('Train')
print(len(train_resampled_data))
print(len(train_resampled_data[train_resampled_data['failure']==1]))
print(len(train_resampled_data[train_resampled_data['failure']==1])/len(train_resampled_data)*100)

print('Test')
print(len(test_resampled_data))
print(len(test_resampled_data[test_resampled_data['failure']==1]))
print(len(test_resampled_data[test_resampled_data['failure']==1])/len(test_resampled_data)*100)

Train
103516
52437
50.655937246416016
Test
1264518
45
0.0035586682040113305


In [15]:
clf = setup(
    data=train_resampled_data,
    target="failure",
    test_data=test_resampled_data,
    session_id=42,
    feature_selection=False,
    use_gpu=True,
    index=False
)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Description,Value
0,Session id,42
1,Target,failure
2,Target type,Binary
3,Original data shape,"(1368034, 9)"
4,Transformed data shape,"(1368034, 9)"
5,Transformed train set shape,"(103516, 9)"
6,Transformed test set shape,"(1264518, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

In [16]:
best_model = compare_models(sort = 'precision')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9994,1.0000,0.9999,0.9990,0.9994,0.9989,0.9989,6.0590
rf,Random Forest Classifier,0.9986,1.0000,0.9994,0.9979,0.9986,0.9972,0.9972,22.2240
dt,Decision Tree Classifier,0.9930,0.9930,0.9926,0.9936,0.9931,0.9860,0.9860,1.3540
knn,K Neighbors Classifier,0.9943,0.9996,0.9978,0.9909,0.9943,0.9885,0.9885,1.5470
xgboost,Extreme Gradient Boosting,0.9667,0.9948,0.9847,0.9513,0.9677,0.9334,0.9340,1.1870
lightgbm,Light Gradient Boosting Machine,0.9415,0.9857,0.9731,0.9166,0.9440,0.8830,0.8847,1.6130
gbc,Gradient Boosting Classifier,0.8077,0.9002,0.8714,0.7764,0.8211,0.6147,0.6195,29.6980
ada,Ada Boost Classifier,0.7290,0.8152,0.8080,0.7022,0.7512,0.4568,0.4625,6.2320
svm,SVM - Linear Kernel,0.5648,0.5852,0.6518,0.5718,0.5818,0.1277,0.1476,0.5270
qda,Quadratic Discriminant Analysis,0.5861,0.6365,0.8975,0.5568,0.6872,0.1653,0.2117,0.5310


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [17]:
final_model = finalize_model(best_model)

In [18]:
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…



---

